In [1]:
import requests

In [2]:
url = "https://www.worldometers.info/coronavirus/country/spain/"

In [3]:
req = requests.get(url)

In [4]:
req

<Response [200]>

In [7]:
from lxml.html import fromstring

In [10]:
root = fromstring(req.text)

In [11]:
root

<Element html at 0x7efbe8302530>

In [72]:
scripts = root.xpath(".//script")

In [227]:
for script in scripts:
    if script.text_content().strip().__contains__("Highcharts.chart"):
        value = script.text_content().strip()
        break

In [228]:
import re

In [229]:
lines = re.compile(r'[\n\t]')

In [230]:
value = lines.sub('', value)

In [231]:
comments = re.compile(r'\/\*(.*?)\*\/')

In [232]:
value = comments.sub('', value)

In [233]:
spaces = re.compile(r'[ ]{2,}')

In [234]:
value = spaces.sub(' ', value)

In [235]:
value = value.replace('\'', '\"')

In [236]:
chart = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )(.*?)(\)\;)')

In [237]:
value = chart.search(value).group(0)

In [238]:
chart_title = re.compile(r'(Highcharts\.chart\(\"[a-zA-z\-]*\"\, )')

In [239]:
value = chart_title.sub('', value)

In [240]:
chart_end = re.compile(r'\)\;')

In [241]:
value = chart_end.sub('', value)

In [242]:
options = re.compile(r'[\w]+\:')

In [243]:
words = options.findall(value)

In [244]:
words = list(set(words))

In [245]:
for word in words:
    value = value.replace(word, '\"' + word.replace(':', '') + '\":')

In [249]:
chart_properties = re.compile(r'(\"chart\"\:)(.*?)(\}\,)')

In [250]:
value = chart_properties.sub('', value)

In [251]:
import json

In [252]:
value = json.loads(value)

In [255]:
value.keys()

dict_keys(['title', 'xAxis', 'yAxis', 'legend', 'credits', 'plotOptions', 'series', 'responsive'])

In [258]:
value['yAxis']['title']['text']

'Currently Infected'

In [259]:
value['xAxis']['categories'][:5]

['Feb 15', 'Feb 16', 'Feb 17', 'Feb 18', 'Feb 19']

In [265]:
value['series'][0]['data'][:5]

[0, 0, 0, 0, 0]